# Arquivo de Execução para Registro de Datasets

* Argumentos:
  * bucket_name: Nome do bucket onde está o arquivo que será utilizado para carregar o Dataset
  * delimiter: Delimitador utilizado no arquivo que será carregado para o Dataset
  * extension: Extensão utilizada no arquivo que será carregado para o Dataset
  * name: Nome do Dataset que será cadastrado
  * path: Caminho onde está o arquivo que será utilizado para carregar o Dataset

Criação dos widgets que receberão os argumentos

In [3]:
dbutils.widgets.text("name", "")
dbutils.widgets.text("bucket_name", "")
dbutils.widgets.text("path", "")
dbutils.widgets.text("delimiter", "")
dbutils.widgets.text("extension", "")

Atribuição dos valores dos argumentos às variáveis

In [5]:
name = dbutils.widgets.get("name")
bucket_name = dbutils.widgets.get("bucket_name")
path = dbutils.widgets.get("path")
delimiter = dbutils.widgets.get("delimiter")
extension = dbutils.widgets.get("extension")

Importação da classe de DatasetBase

In [7]:
%run ../Datasets/DatasetBase

Importação da biblioteca Utils

In [9]:
%run ../Utils/Utils

* Criação do Bucket que armanezará o arquivo que irá manter os cadastros dos Datasets.
* Verificação se o Dataset já está cadastrado
* Cadastro do novo Dataset se ainda não houver

In [11]:
bucket_write = dbutils.secrets.get(scope = "aws", key = "aws-bucket-name")
DatasetBase.createBucket(bucket_write)
count = 1

complete_path = f'/mnt/{bucket_write}/raw/datasets'

from datetime import datetime
from pyspark.sql.functions import col, max

if file_exists(complete_path):
    oldDataFrame = DatasetBase.loadDatabase(bucket_write, complete_path)
    
    if oldDataFrame.filter(col('name') == name).count() < 1:
        count = oldDataFrame.agg(max(col('id'))).collect()[0][0] + 1  
        conf = [(count, name, bucket_name, path, delimiter, extension, datetime.now().strftime("%Y-%m-%dT%H:%M:%S.000Z"))]
        newDataFrame = spark.createDataFrame(conf, ['id', 'name', 'bucket_name', 'path', 'delimiter', 'extension', 'created_at'])
        df = newDataFrame.union(oldDataFrame)
    else:
        dbutils.notebook.exit('Dataset já cadastrado!!')
else:
    count = 1
    conf = [(count, name, bucket_name, path, delimiter, extension, datetime.now().strftime("%Y-%m-%dT%H:%M:%S.000Z"))]
    df = spark.createDataFrame(conf, ['id', 'name', 'bucket_name', 'path', 'delimiter', 'extension', 'created_at'])

try:
    DatasetBase.writeDataset(df, bucket_write, complete_path)
    print('Dataset cadastrado com Sucesso!!!')
except Exception:
    dbutils.notebook.exit('ERROR: Erro na escrita do Dataset')

Exibição do Arquivo com todos os Datasets Cadastrados

In [13]:
df = DatasetBase.loadDatabase(bucket_name, complete_path)
display(df)